# Transfer Learning

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

In [2]:
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [3]:
# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

In [4]:
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [5]:
# loading the Fashion-MNIST dataset
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

# trimming the data since it takes lot of time
X_train_full = X_train_full[:30000]
y_train_full = y_train_full[:30000]

X_test = X_test[:5000]
y_test = y_test[:5000]

# scaling the dataset
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0

# dividing the dataset into traingin and validation set
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [6]:
# defining the dataset
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)

X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [7]:
X_train_A.shape # checking shape of training set A

(19875, 28, 28)

In [8]:
X_train_B.shape # checking shape of training set B

(200, 28, 28)

In [9]:
y_train_A[:30] # checking first 30 y-labels for training set A

array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4, 0, 1, 6, 3, 4, 3, 2, 6, 5, 3, 4, 5,
       1, 3, 4, 2, 0, 6, 7, 1], dtype=uint8)

In [10]:
y_train_B[:30] # checking first 30 y-labels for training set B

array([1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.], dtype=float32)

In [11]:
tf.random.set_seed(42)
np.random.seed(42)

In [12]:
# defining the model
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [13]:
# compiling the model
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

/usr/local/anaconda/envs/tensorflow2/lib/python3.6/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [14]:
# training the model
history = model_A.fit(X_train_A, y_train_A, epochs=5,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/5
622/622 [==============================] - 3s 3ms/step - loss: 0.7979 - accuracy: 0.7498 - val_loss: 0.4762 - val_accuracy: 0.8490
Epoch 2/5
622/622 [==============================] - 2s 3ms/step - loss: 0.4260 - accuracy: 0.8582 - val_loss: 0.3921 - val_accuracy: 0.8652
Epoch 3/5
622/622 [==============================] - 2s 3ms/step - loss: 0.3702 - accuracy: 0.8737 - val_loss: 0.3700 - val_accuracy: 0.8712
Epoch 4/5
622/622 [==============================] - 2s 3ms/step - loss: 0.3418 - accuracy: 0.8812 - val_loss: 0.3285 - val_accuracy: 0.8869
Epoch 5/5
622/622 [==============================] - 2s 3ms/step - loss: 0.3240 - accuracy: 0.8869 - val_loss: 0.3229 - val_accuracy: 0.8884


In [15]:
model_A.save("my_model_A.h5") # saving the model we created

In [16]:
# defining the model
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [17]:
# compiling the model with binary crossentropy
# that can accept either logits (i.e values from last linear node, z)
# or probabilities from the last Sigmoid node
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

/usr/local/anaconda/envs/tensorflow2/lib/python3.6/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [18]:
# training the model
history = model_B.fit(X_train_B, y_train_B, epochs=5,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/5
7/7 [==============================] - 1s 39ms/step - loss: 0.9573 - accuracy: 0.4650 - val_loss: 0.6314 - val_accuracy: 0.6004
Epoch 2/5
7/7 [==============================] - 0s 14ms/step - loss: 0.5692 - accuracy: 0.7450 - val_loss: 0.4784 - val_accuracy: 0.8529
Epoch 3/5
7/7 [==============================] - 0s 13ms/step - loss: 0.4503 - accuracy: 0.8650 - val_loss: 0.4102 - val_accuracy: 0.8945
Epoch 4/5
7/7 [==============================] - 0s 13ms/step - loss: 0.3879 - accuracy: 0.8950 - val_loss: 0.3647 - val_accuracy: 0.9178
Epoch 5/5
7/7 [==============================] - 0s 13ms/step - loss: 0.3435 - accuracy: 0.9250 - val_loss: 0.3300 - val_accuracy: 0.9320


In [19]:
model_A.summary() # generating model summary

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 4

In [20]:
model_B.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 300)               235500    
_________________________________________________________________
dense_7 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_8 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_9 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_10 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                

In [22]:
'test\n'[:-1] # to omit the last character

'test'

In [23]:
model_A = keras.models.load_model("my_model_A.h5") # loading our saved model
model_B_on_A = keras.models.Sequential(model_A.layers[:-1]) # creating new model based on existing layer, just omit the last layer
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid")) # adding new layer to new model

In [24]:
# model_A and model_B_on_A now share some layers. When you train
# model_B_on_A, it will also affect model_A. To avoid that, you need to clone
# model_A before you reuse its layers. To do this, you clone model A’s
# architecture with clone_model(), then copy its weights
# IMP: (since clone_model() does not clone the weights)

model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [25]:
# freezing reused layers [it will behave like a non linear equation here, just perfect weights are there!]
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

# compiling the model
model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])

In [26]:
# training the model
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

# unfreezing reused layers
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

# compiling after reducing learning rate to avoid damaging the reused weights
model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])

# training the model
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=5,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 1s 38ms/step - loss: 0.4927 - accuracy: 0.7700 - val_loss: 0.5257 - val_accuracy: 0.7099
Epoch 2/4
7/7 [==============================] - 0s 12ms/step - loss: 0.4738 - accuracy: 0.7850 - val_loss: 0.5051 - val_accuracy: 0.7323
Epoch 3/4
7/7 [==============================] - 0s 12ms/step - loss: 0.4536 - accuracy: 0.8050 - val_loss: 0.4873 - val_accuracy: 0.7485
Epoch 4/4
7/7 [==============================] - 0s 12ms/step - loss: 0.4360 - accuracy: 0.8150 - val_loss: 0.4709 - val_accuracy: 0.7657
Epoch 1/5
7/7 [==============================] - 1s 39ms/step - loss: 0.3800 - accuracy: 0.8550 - val_loss: 0.3594 - val_accuracy: 0.8631
Epoch 2/5
7/7 [==============================] - 0s 14ms/step - loss: 0.2878 - accuracy: 0.9150 - val_loss: 0.2822 - val_accuracy: 0.9138
Epoch 3/5
7/7 [==============================] - 0s 13ms/step - loss: 0.2243 - accuracy: 0.9550 - val_loss: 0.2359 - val_accuracy: 0.9391
Epoch 4/5
7/7 [===================

In [27]:
model_B.evaluate(X_test_B, y_test_B) # evaluating the model A

31/31 [==============================] - 0s 2ms/step - loss: 0.3261 - accuracy: 0.9328


[0.32605302333831787, 0.9327818155288696]

In [28]:
model_B_on_A.evaluate(X_test_B, y_test_B) # evaluating the model B

31/31 [==============================] - 0s 2ms/step - loss: 0.1745 - accuracy: 0.9731


[0.17450204491615295, 0.9731127023696899]

In [29]:
(100 - 96.95) / (100 - 99.25)

4.066666666666663